## Clone, install, import

In [ ]:
#!git clone #add link

In [2]:
%pip install -r /home/arjan_v_d/planimals/requirements.txt

import fastparquet
import shutil

## Download

In [1]:
%run /home/arjan_v_d/planimals/src/utils.py
create_if_absent("../data/dbnl_xml")
#download_and_unzip(url = "https://www.dbnl.org/letterkunde/pd/xml_pd.zip", output_dir = "../data/dbnl_xml")

Directory ../data/dbnl_xml created


## Preprocess

In [37]:
%run /home/arjan_v_d/planimals/src/preprocessing.py

html_to_decimal_dict = make_html_entity_dict()
addition =  additional_declaration_str(html_to_decimal_dict, '.dtd"')
add_declaration_to_xml('../data/dbnl_xml', '<!DOCTYPE', '.dtd"', addition)

In [38]:
create_if_absent("../data/dbnl_txt")
dbnl_to_txt("../data/dbnl_xml", "../data/dbnl_txt")

Directory ../data/dbnl_txt already exists


In [39]:
create_if_absent("../data/dbnl_folia")
txt_to_folia("../data/dbnl_txt", "../data/dbnl_folia")

Directory ../data/dbnl_folia already exists


Looping through text files: 100%|██████████| 1/1 [00:00<00:00, 3233.85it/s]


In [40]:
create_if_absent("../data/dbnl_sentences")
folia_files = select_all_files("../data/dbnl_folia", "folia.xml")

for f in tqdm.tqdm(folia_files, desc = "making df"):
  a  = folia_to_df(f)


Directory ../data/dbnl_sentences already exists


making df: 100%|██████████| 1/1 [00:21<00:00, 21.25s/it]

3186
2925


In [ ]:
# 3 things: text in sentence, dutch, footnotes. Add that!

In [4]:
def count_files_in_folder(folder_path):
    file_count = 0

    # Iterate over all the items in the folder
    for _, _, files in os.walk(folder_path):
        file_count += len(files)

    return file_count

# Provide the path to the folder you want to count files in
folder_paths = ['dbnl_xml', 'dbnl_txt', 'dbnl_folia', 'dbnl_sentences']

for p in folder_paths:
    num_files = count_files_in_folder(p)
    print(f"Number of files in '{p}': {num_files}")


Number of files in 'dbnl_xml': 3433
Number of files in 'dbnl_txt': 3415
Number of files in 'dbnl_folia': 3422
Number of files in 'dbnl_sentences': 3413


In [1]:
subfolders1 = ["towards_big_df/subfolder_1", "towards_big_df/subfolder_2", "towards_big_df/subfolder_3", "towards_big_df/subfolder_4", "towards_big_df/subfolder_5"]
subfolders2 = ["towards_big_df/subfolder_5", "towards_big_df/subfolder_6", "towards_big_df/subfolder_7"]


In [ ]:
def select_all_files(path_to_folder, file_extension):
  return [os.path.join(path_to_folder, f) for f in os.listdir(path_to_folder) if f.endswith(file_extension)]

n = 0
for s in subfolders1:
    n+=1
    paths = select_all_files(s, ".pkl")
    dfs = []
    for p in tqdm.tqdm(paths, desc = "reading pkl's"):
        df = pd.read_pickle(p)
        dfs.append(df)
    total_df = pd.concat(dfs, ignore_index = True)
    total_df.to_pickle(f"towards_big_df_sub_{n}")

reading pkl's: 100%|██████████| 500/500 [00:13<00:00, 38.28it/s] 

In [2]:
n = 5
for s in subfolders2:
    n+=1
    paths = select_all_files(s, ".pkl")
    dfs = []
    for p in tqdm.tqdm(paths, desc = "reading pkl's"):
        df = pd.read_pickle(p)
        dfs.append(df)
    total_df = pd.concat(dfs, ignore_index = True)
    total_df.to_pickle(f"towards_big_df_sub_{n}")

reading pkl's: 100%|██████████| 413/413 [00:11<00:00, 36.02it/s]


In [1]:
# Define the source folder containing the files
source_folder = "dbnl_sentences"

# Define the destination folder for the subfolders
destination_folder = "towards_big_df"

# Create the destination folder if it doesn't exist
os.makedirs(destination_folder, exist_ok=True)

# Get a list of all the files in the source folder
files = os.listdir(source_folder)

# Calculate the number of files per subfolder
files_per_subfolder = 500

# Calculate the total number of subfolders needed
total_subfolders = len(files) // files_per_subfolder + 1

# Create the subfolders and move the files into them
for i in range(total_subfolders):
    # Create a subfolder
    subfolder_path = os.path.join(destination_folder, f'subfolder_{i+1}')
    os.makedirs(subfolder_path, exist_ok=True)
    
    # Move the files into the subfolder
    start_index = i * files_per_subfolder
    end_index = start_index + files_per_subfolder
    for file in files[start_index:end_index]:
        file_path = os.path.join(source_folder, file)
        shutil.move(file_path, subfolder_path)


In [7]:
df1 = "towards_big_df_sub_1"
df2 = "towards_big_df_sub_2"
df3 = "towards_big_df_sub_3"
df4 = "towards_big_df_sub_4"
df5 = "towards_big_df_sub_5"
df6 = "towards_big_df_sub_6"
df7 = "towards_big_df_sub_7"


In [8]:
e = df5
f = "df5" 
e_df = pd.read_pickle(e)
e_df.to_parquet(f'parquets/{f}.parquet')

In [ ]:
merged_df = pd.concat([
    pd.read_parquet('parquets/df1.parquet'),
    pd.read_parquet('parquets/df2.parquet'),
    pd.read_parquet('parquets/df3.parquet'),
    pd.read_parquet('parquets/df4.parquet'),
    pd.read_parquet('parquets/df5.parquet'),
    pd.read_parquet('parquets/df6.parquet'),
    pd.read_parquet('parquets/df7.parquet')],
    ignore_index=True)


In [4]:
merged_df = pd.concat([
    pd.read_parquet('parquets/df1.parquet'),
    pd.read_parquet('parquets/df2.parquet')],
    ignore_index=True)

In [ ]:
merged_df.to_parquet("parquets/df1and2.parquet")

In [ ]:
import pandas as pd
a = pd.read_parquet("parquets/df1and2.parquet")